In [1]:
import logging
import numpy as np

from wave_monitor import WaveMonitor
h = logging.StreamHandler()
h.setLevel(logging.DEBUG)
h.setFormatter(logging.Formatter("%(levelname)s %(message)s"))
WaveMonitor.logger.addHandler(h)

monitor = WaveMonitor()

t = np.linspace(0, 1, 1_000_001)  # 1m pts ~= 1ms for 1GSa/s.
f = 5
i_wave = np.cos(2 * np.pi * f * t)
q_wave = np.sin(2 * np.pi * f * t)

WaveMonitor.logger.setLevel(logging.DEBUG)
monitor.add_wfm("wave_1", t, [i_wave, q_wave])
WaveMonitor.logger.setLevel(logging.INFO)

DEBUG query received: {'_type': 'wfm_interval', 'interval': 0}
DEBUG Adding waveform 'wave_1'


In [2]:
%timeit -n 3 -r 5 monitor.add_wfm("wave_1", t, [i_wave, q_wave], dtype=None)

The slowest run took 6.36 times longer than the fastest. This could mean that an intermediate result is being cached.
6.39 μs ± 6.03 μs per loop (mean ± std. dev. of 5 runs, 3 loops each)


In [3]:
%timeit -n 3 -r 5 monitor.add_wfm("wave_1", t, [i_wave, q_wave], dtype=np.float32)

7.46 μs ± 4.73 μs per loop (mean ± std. dev. of 5 runs, 3 loops each)


In [4]:
%timeit -n 3 -r 5 monitor.get_wfm_interval()

The slowest run took 14.00 times longer than the fastest. This could mean that an intermediate result is being cached.
507 ns ± 681 ns per loop (mean ± std. dev. of 5 runs, 3 loops each)


In [5]:
mon2 = WaveMonitor()
mon2.get_wfm_interval = lambda: 0  # patch to avoid the delay.
%timeit -n 3 -r 5 mon2.add_wfm("wave_2", t, [i_wave, q_wave])

The slowest run took 5.11 times longer than the fastest. This could mean that an intermediate result is being cached.
8.65 μs ± 6.88 μs per loop (mean ± std. dev. of 5 runs, 3 loops each)


C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py:409: UserWarning: Monitor is already running, not starting a new one.
  warnings.warn("Monitor is already running, not starting a new one.")


In [6]:
from PySide6.QtNetwork import QLocalSocket
class FakeSocket:
    def write(self, data: bytes) -> int:
        return len(data)
    
    def waitForBytesWritten(self) -> bool:
        return True
    
    def state(self):
        return QLocalSocket.ConnectedState
    
mon2.sock = FakeSocket()  # patch to avoid the delay.
%timeit -n 3 -r 5 mon2.add_wfm("wave_2", t, [i_wave, q_wave])

The slowest run took 4.05 times longer than the fastest. This could mean that an intermediate result is being cached.
7.31 μs ± 5.25 μs per loop (mean ± std. dev. of 5 runs, 3 loops each)


IO worker op failed: write
Traceback (most recent call last):
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 83, in run
    self._handle_write(payload["msg"])  # fire-and-forget
    ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 183, in _handle_write
    self._write_payload(payload)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 154, in _write_payload
    raise RuntimeError("Failed to write to socket.")
RuntimeError: Failed to write to socket.


IO worker op failed: write
Traceback (most recent call last):
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 83, in run
    self._handle_write(payload["msg"])  # fire-and-forget
    ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 183, in _handle_write
    self._write_payload(payload)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 154, in _write_payload
    raise RuntimeError("Failed to write to socket.")
RuntimeError: Failed to write to socket.


In [7]:
%timeit -n 3 -r 5 monitor.add_wfm("wave_1", t, [i_wave, q_wave])

The slowest run took 8.10 times longer than the fastest. This could mean that an intermediate result is being cached.
10.3 μs ± 11.6 μs per loop (mean ± std. dev. of 5 runs, 3 loops each)


IO worker op failed: write
Traceback (most recent call last):
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 83, in run
    self._handle_write(payload["msg"])  # fire-and-forget
    ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 183, in _handle_write
    self._write_payload(payload)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 154, in _write_payload
    raise RuntimeError("Failed to write to socket.")
RuntimeError: Failed to write to socket.
IO worker op failed: write
Traceback (most recent call last):
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 83, in run
    self._handle_write(payload["msg"])  # fire-and-forget
    ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 183, in _handle_write
    self._write_payload(payload)
    ~

IO worker op failed: write
Traceback (most recent call last):
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 83, in run
    self._handle_write(payload["msg"])  # fire-and-forget
    ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 183, in _handle_write
    self._write_payload(payload)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 154, in _write_payload
    raise RuntimeError("Failed to write to socket.")
RuntimeError: Failed to write to socket.
IO worker op failed: write
Traceback (most recent call last):
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 83, in run
    self._handle_write(payload["msg"])  # fire-and-forget
    ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^
  File "C:\Users\qiujv\Downloads\WaveMonitor\src\wave_monitor\client.py", line 183, in _handle_write
    self._write_payload(payload)
    ~

In [ ]:
from viztracer import VizTracer
with VizTracer(output_file="result.json") as tracer:
    monitor.add_wfm("wave_1", t, [i_wave, q_wave], dtype=np.float16)

Total Entries: 123                                                              
Use the following command to open the report:
vizviewer c:\Users\qiujv\dev\WaveMonitor\notebooks\result.json


In [ ]:
# close the window now.
%timeit -n 3 -r 5 monitor.add_wfm("wave_1", t, [i_wave, q_wave])

35.1 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
monitor.refresh_connect()

True

In [ ]:
monitor.sock.state()

<LocalSocketState.ConnectedState: 3>

In [ ]:
from wave_monitor import proto
msg = dict(_type="add_wfm", name="wave_1", t=t.astype(np.float16), ys=[i_wave.astype(np.float16), q_wave.astype(np.float16)])
payload = proto.encode(msg)
print(f"Payload size: {len(payload)/1e6:.2f} MB")

%timeit -n 3 -r 5 proto.encode(msg)

Payload size: 6.00 MB
3.77 ms ± 179 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit i_wave.astype(np.float32)

In [ ]:
%timeit i_wave.astype(np.float16)

In [ ]:
%timeit i_wave.astype(np.float64)

In [ ]:
%timeit i_wave